# PART 1: PROCESSING DATASETS AND CLEANING DATA

In [1]:
#IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statistics import mean, median, quantiles, variance
import datetime
from sklearn.model_selection import train_test_split
from kneed import KneeLocator
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression

import pickle
import unicodedata
import shap

C:\Users\mirei\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# detect negative values in consumption
def detect_neg_values(data, column):
    count_neg = 0
    for row in data[column]:
        if row <0:
            count_neg+=1
    return count_neg

In this part we have five blocks depending on the datasets (consumption by sector, postal code, district, activities or census section)

## BLOCK 1: TEMPERATURE, COMERCIAL, DOMESTIC AND INDUSTRIAL DATASETS

We have data of water consumption in different areas of Catalonia separated by the use of this water: Domestic, Industrial and Comercial.

Now we will focus on datasets in Barcelona with a domestic, industrial and comercial use. The datasets have 4 columns corresponding to Date, Use, Consumption and Id

The Date column is the date of the consumption register of the data. The Use column corresponds to the use (Domestic, Industrial and Comercial) of the water consumption. The Consumption column is the water consumption and the Id is the counter identifier used to measure the water flow of each consumer.

First we are going to process the datasets and clean the data.

### TEMPERATURE DATASET

In [3]:
# open TEMPERATURE data in barcelona https://opendata-ajuntament.barcelona.cat/data/en/dataset/temperatures-hist-bcn
temperature_barcelona = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/temperature.csv", sep=",")

In [4]:
temperature_barcelona

,Any,Mes,Desc_Mes,Temperatura
0,1780,1,Gener,6.7
1,1780,2,Febrer,7.3
2,1780,3,Març,11.6
3,1780,4,Abril,11.4
4,1780,5,Maig,16.3
...,...,...,...,...
2899,2021,8,Agost,24.5
2900,2021,9,Setembre,23.0
2901,2021,10,Octubre,18.1
2902,2021,11,Novembre,11.3


In [5]:
# delete null rows
temperature_barcelona=temperature_barcelona.dropna(how='any', axis=0)

In [6]:
temperature_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2904 entries, 0 to 2903
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Any          2904 non-null   int64  
 1   Mes          2904 non-null   int64  
 2   Desc_Mes     2904 non-null   object 
 3   Temperatura  2904 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 90.9+ KB


### DOMESTIC DATASET

In [140]:
# open domestic data in barcelona
data_domestic_barcelona = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/water_consumption/domestic_barcelona.csv", sep=";")

In [141]:
# delete null rows
data_domestic_barcelona=data_domestic_barcelona.dropna(how='any', axis=0)

# append new column LOCATION
data_domestic_barcelona['LOCATION'] = ['BARCELONA' for i in range(len(data_domestic_barcelona))]

# convert date object to datetime
data_domestic_barcelona['FECHA'] = pd.to_datetime(data_domestic_barcelona['FECHA'], infer_datetime_format=True)

# append year column
data_domestic_barcelona['YEAR'] = [date.strftime("%Y") for date in data_domestic_barcelona['FECHA']]

# append month column
data_domestic_barcelona['MONTH'] = [date.month for date in data_domestic_barcelona['FECHA']]

Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.


In [142]:
# append covid state column
# AFTER COVID 0
# BEFORE COVID 1
# DURING COVID 2
data_domestic_barcelona['COVID_STATE'] = [(0 if data_domestic_barcelona['YEAR'].iloc[row] =='2021' else (1 if data_domestic_barcelona['YEAR'].iloc[row] == '2019' else 2)) for row in range(len(data_domestic_barcelona['YEAR'])) ]

In [143]:
data_domestic_barcelona

,FECHA,USO,CONSUMO,ID_CONTADOR,LOCATION,YEAR,MONTH,COVID_STATE
0,2019-08-03,DOMÈSTIC,164,8213,BARCELONA,2019,8,1
1,2019-10-02,DOMÈSTIC,173,9368,BARCELONA,2019,10,1
2,2019-01-19,DOMÈSTIC,178,5564,BARCELONA,2019,1,1
3,2019-02-17,DOMÈSTIC,498,7294,BARCELONA,2019,2,1
4,2019-03-02,DOMÈSTIC,190,5022,BARCELONA,2019,3,1
...,...,...,...,...,...,...,...,...
1048570,2019-06-21,DOMÈSTIC,287,10232,BARCELONA,2019,6,1
1048571,2019-08-02,DOMÈSTIC,162,7121,BARCELONA,2019,8,1
1048572,2019-05-29,DOMÈSTIC,53,6038,BARCELONA,2019,5,1
1048573,2019-09-23,DOMÈSTIC,506,7897,BARCELONA,2019,9,1


In [144]:
data_domestic_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   FECHA        1048575 non-null  datetime64[ns]
 1   USO          1048575 non-null  object        
 2   CONSUMO      1048575 non-null  int64         
 3   ID_CONTADOR  1048575 non-null  int64         
 4   LOCATION     1048575 non-null  object        
 5   YEAR         1048575 non-null  object        
 6   MONTH        1048575 non-null  int64         
 7   COVID_STATE  1048575 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 64.0+ MB


In [145]:
# types of 'uso'
print('Type of use: ', data_domestic_barcelona['USO'].unique())

# all domestic -> good

print('There are ', len(data_domestic_barcelona['FECHA'].unique()), ' dates in the dataset')

Type of use:  ['DOMÈSTIC']
There are  1079  dates in the dataset


In [146]:
# detect negative values of consumption
count_neg = detect_neg_values(data_domestic_barcelona, 'CONSUMO')
print('There are ', count_neg, ' negative values in consumption')
print('There are a total of ', len(data_domestic_barcelona), 'values of consumption')
print((count_neg/len(data_domestic_barcelona)) *100)

# so we can delete the negative values -> small part

There are  1682  negative values in consumption
There are a total of  1048575 values of consumption
0.16040817299668597


In [147]:
data_domestic_barcelona = data_domestic_barcelona[data_domestic_barcelona['CONSUMO']>=0]

### COMERCIAL DATASET

In [148]:
# open comercial data in barcelona
data_comercial_barcelona = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/water_consumption/commercial_barcelona.csv", sep=";")

In [149]:
# delete null rows
data_comercial_barcelona=data_comercial_barcelona.dropna(how='any', axis=0)

# append new column LOCATION
data_comercial_barcelona['LOCATION'] = ['BARCELONA' for i in range(len(data_comercial_barcelona))]

# convert date object to datetime
data_comercial_barcelona['FECHA'] = pd.to_datetime(data_comercial_barcelona['FECHA'], infer_datetime_format=True)

# append year column
data_comercial_barcelona['YEAR'] = [date.strftime("%Y") for date in data_comercial_barcelona['FECHA']]

# append month column
data_comercial_barcelona['MONTH'] = [date.month for date in data_comercial_barcelona['FECHA']]

Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.


In [150]:
# append a new column covid_state
# AFTER COVID 0
# BEFORE COVID 1
# DURING COVID 2
data_comercial_barcelona['COVID_STATE'] = [(0 if data_comercial_barcelona['YEAR'].iloc[row] =='2021' else (1 if data_comercial_barcelona['YEAR'].iloc[row] == '2019' else 2)) for row in range(len(data_comercial_barcelona['YEAR'])) ]

In [151]:
data_comercial_barcelona

,FECHA,USO,CONSUMO,ID_CONTADOR,LOCATION,YEAR,MONTH,COVID_STATE
0,2019-02-01,COMERCIAL,86,7630,BARCELONA,2019,2,1
1,2019-03-21,COMERCIAL,413,8191,BARCELONA,2019,3,1
2,2019-06-02,COMERCIAL,22,8675,BARCELONA,2019,6,1
3,2019-01-13,COMERCIAL,82,5204,BARCELONA,2019,1,1
4,2019-05-02,COMERCIAL,2,14835,BARCELONA,2019,5,1
...,...,...,...,...,...,...,...,...
1048570,2019-12-19,COMERCIAL,150,11514,BARCELONA,2019,12,1
1048571,2020-01-21,COMERCIAL,101,926,BARCELONA,2020,1,2
1048572,2019-02-12,COMERCIAL,1265,4723,BARCELONA,2019,2,1
1048573,2020-03-30,COMERCIAL,1,1076,BARCELONA,2020,3,2


In [152]:
data_comercial_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   FECHA        1048575 non-null  datetime64[ns]
 1   USO          1048575 non-null  object        
 2   CONSUMO      1048575 non-null  int64         
 3   ID_CONTADOR  1048575 non-null  int64         
 4   LOCATION     1048575 non-null  object        
 5   YEAR         1048575 non-null  object        
 6   MONTH        1048575 non-null  int64         
 7   COVID_STATE  1048575 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 64.0+ MB


In [153]:
# types of 'uso'
print('Type of use: ', data_comercial_barcelona['USO'].unique())

# all comercial -> good

print('There are ', len(data_comercial_barcelona['FECHA'].unique()), ' dates in the dataset')

Type of use:  ['COMERCIAL']
There are  1079  dates in the dataset


In [154]:
# detect negative values of consumption
count_neg = detect_neg_values(data_comercial_barcelona, 'CONSUMO')
print('There are ', count_neg, ' negative values in consumption')
print('There are a total of ', len(data_comercial_barcelona), 'values of consumption')
print((count_neg/len(data_comercial_barcelona)) *100)

# so we can delete the negative values -> small part

There are  2415  negative values in consumption
There are a total of  1048575 values of consumption
0.23031256705528932


In [155]:
data_comercial_barcelona = data_comercial_barcelona[data_comercial_barcelona['CONSUMO']>=0]

### INDUSTRIAL DATASET

In [156]:
# open industrial data in barcelona
data_industrial_barcelona = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/water_consumption/industrial_barcelona.csv", sep = ';')

In [157]:
# delete null rows
data_industrial_barcelona=data_industrial_barcelona.dropna(how='any', axis=0)

# append new column LOCATION
data_industrial_barcelona['LOCATION'] = ['BARCELONA' for i in range(len(data_industrial_barcelona))]

# convert date object to datetime
data_industrial_barcelona['FECHA'] = pd.to_datetime(data_industrial_barcelona['FECHA'], infer_datetime_format=True)

# append year column
data_industrial_barcelona['YEAR'] = [date.strftime("%Y") for date in data_industrial_barcelona['FECHA']]

# append month column
data_industrial_barcelona['MONTH'] = [date.month for date in data_industrial_barcelona['FECHA']]

In [158]:
# append covid state column
# AFTER COVID 0
# BEFORE COVID 1
# DURING COVID 2
data_industrial_barcelona['COVID_STATE'] = [(0 if data_industrial_barcelona['YEAR'].iloc[row] =='2021' else (1 if data_industrial_barcelona['YEAR'].iloc[row] == '2019' else 2)) for row in range(len(data_industrial_barcelona['YEAR'])) ]

In [159]:
data_industrial_barcelona

,FECHA,USO,CONSUMO,ID_CONTADOR,LOCATION,YEAR,MONTH,COVID_STATE
0,2019-02-14,INDUSTRIAL,2,838,BARCELONA,2019,2,1
1,2019-02-14,INDUSTRIAL,79,1207,BARCELONA,2019,2,1
2,2019-02-17,INDUSTRIAL,12,1135,BARCELONA,2019,2,1
3,2019-01-06,INDUSTRIAL,31,1286,BARCELONA,2019,1,1
4,2019-01-06,INDUSTRIAL,184,947,BARCELONA,2019,1,1
...,...,...,...,...,...,...,...,...
1048570,2019-12-02,INDUSTRIAL,123,1209,BARCELONA,2019,12,1
1048571,2019-12-16,INDUSTRIAL,7183,1874,BARCELONA,2019,12,1
1048572,2020-02-17,INDUSTRIAL,481,2301,BARCELONA,2020,2,2
1048573,2020-07-17,INDUSTRIAL,54,2372,BARCELONA,2020,7,2


In [160]:
data_industrial_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   FECHA        1048575 non-null  datetime64[ns]
 1   USO          1048575 non-null  object        
 2   CONSUMO      1048575 non-null  int64         
 3   ID_CONTADOR  1048575 non-null  int64         
 4   LOCATION     1048575 non-null  object        
 5   YEAR         1048575 non-null  object        
 6   MONTH        1048575 non-null  int64         
 7   COVID_STATE  1048575 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 64.0+ MB


In [161]:
# types of 'uso'
print('Type of use: ', data_industrial_barcelona['USO'].unique())

# all industrial -> good

print('There are ', len(data_industrial_barcelona['FECHA'].unique()), ' dates in the dataset')

Type of use:  ['INDUSTRIAL']
There are  1079  dates in the dataset


In [162]:
# detect negative values of consumption
count_neg = detect_neg_values(data_industrial_barcelona, 'CONSUMO')
print('There are ', count_neg, ' negative values in consumption')
print('There are a total of ', len(data_industrial_barcelona), 'values of consumption')
print((count_neg/len(data_industrial_barcelona)) *100)

# so we can delete the negative values -> small part

There are  2225  negative values in consumption
There are a total of  1048575 values of consumption
0.21219273776315475


In [163]:
data_industrial_barcelona = data_industrial_barcelona[data_industrial_barcelona['CONSUMO']>=0]

#### Merge datasets

Merge datasets to have a unique with all the information about water consumption in Barcelona.

In [164]:
data_barcelona = pd.concat([data_industrial_barcelona, data_comercial_barcelona, data_domestic_barcelona], axis=0)

In [165]:
data_barcelona

,FECHA,USO,CONSUMO,ID_CONTADOR,LOCATION,YEAR,MONTH,COVID_STATE
0,2019-02-14,INDUSTRIAL,2,838,BARCELONA,2019,2,1
1,2019-02-14,INDUSTRIAL,79,1207,BARCELONA,2019,2,1
2,2019-02-17,INDUSTRIAL,12,1135,BARCELONA,2019,2,1
3,2019-01-06,INDUSTRIAL,31,1286,BARCELONA,2019,1,1
4,2019-01-06,INDUSTRIAL,184,947,BARCELONA,2019,1,1
...,...,...,...,...,...,...,...,...
1048570,2019-06-21,DOMÈSTIC,287,10232,BARCELONA,2019,6,1
1048571,2019-08-02,DOMÈSTIC,162,7121,BARCELONA,2019,8,1
1048572,2019-05-29,DOMÈSTIC,53,6038,BARCELONA,2019,5,1
1048573,2019-09-23,DOMÈSTIC,506,7897,BARCELONA,2019,9,1


In [166]:
data_barcelona['YEAR'].unique()

array(['2019', '2020', '2021'], dtype=object)

In [167]:
data_barcelona['USO'].value_counts()

DOMÈSTIC      1046893
INDUSTRIAL    1046350
COMERCIAL     1046160
Name: USO, dtype: int64

In [168]:
# convert object type year to int
data_barcelona['YEAR'] = data_barcelona['YEAR'].astype(int)

In [169]:
# append a new column named 'CONSUMO_COVID' that represents 1 when de consumption is higher than the mean consumption 
# during covid and 0 when de consumption is lower than the mean consumption during covid
consum_covid = [data_barcelona['CONSUMO'].iloc[covid_status] for covid_status in range(len(data_barcelona['COVID_STATE'])) if data_barcelona['COVID_STATE'].iloc[covid_status] == 2]
mean_covid = mean(consum_covid)

consumption_covid = []
for row in range(len(data_barcelona)):
    if data_barcelona['CONSUMO'].iloc[row] > mean_covid:
        consumption_covid.append(1)
    if data_barcelona['CONSUMO'].iloc[row] <= mean_covid:
        consumption_covid.append(0)
    
data_barcelona['CONSUMO_COVID'] = consumption_covid

In [170]:
# remove 'DOMESTIC' accent
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

data_barcelona['USO'] = [strip_accents(data_barcelona['USO'].iloc[row]) for row in range(len(data_barcelona['USO']))]

In [171]:
# mapping USO to int
# INDUSTRIAL = 0, DOMESTIC = 1, COMERCIAL = 2
data_barcelona['USO'] = [(0 if data_barcelona['USO'].iloc[row] =='INDUSTRIAL' else (1 if data_barcelona['USO'].iloc[row] == 'DOMESTIC' else 2)) for row in range(len(data_barcelona['USO'])) ]

In [172]:
# append new column TEMPERATURE
data_barcelona = data_barcelona.merge(temperature_barcelona, left_on=['MONTH', 'YEAR'], right_on=['Mes', 'Any'])
data_barcelona = data_barcelona.drop(['Any', 'Mes'], axis=1)

In [173]:
# rename column names
data_barcelona.rename(columns = {'Desc_Mes':'MONTH_LONG', 'Temperatura':'TEMPERATURE'}, inplace = True)

In [174]:
data_barcelona

,FECHA,USO,CONSUMO,ID_CONTADOR,LOCATION,YEAR,MONTH,COVID_STATE,CONSUMO_COVID,MONTH_LONG,TEMPERATURE
0,2019-02-14,0,2,838,BARCELONA,2019,2,1,0,Febrer,11.9
1,2019-02-14,0,79,1207,BARCELONA,2019,2,1,0,Febrer,11.9
2,2019-02-17,0,12,1135,BARCELONA,2019,2,1,0,Febrer,11.9
3,2019-02-03,0,113,1328,BARCELONA,2019,2,1,0,Febrer,11.9
4,2019-02-11,0,22,1258,BARCELONA,2019,2,1,0,Febrer,11.9
...,...,...,...,...,...,...,...,...,...,...,...
3139398,2021-12-12,1,429,18780,BARCELONA,2021,12,0,0,Desembre,10.9
3139399,2021-12-13,1,311,7935,BARCELONA,2021,12,0,0,Desembre,10.9
3139400,2021-12-13,1,332,14312,BARCELONA,2021,12,0,0,Desembre,10.9
3139401,2021-12-14,1,821,14312,BARCELONA,2021,12,0,0,Desembre,10.9


In [175]:
data_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3139403 entries, 0 to 3139402
Data columns (total 11 columns):
 #   Column         Dtype         
---  ------         -----         
 0   FECHA          datetime64[ns]
 1   USO            int64         
 2   CONSUMO        int64         
 3   ID_CONTADOR    int64         
 4   LOCATION       object        
 5   YEAR           int32         
 6   MONTH          int64         
 7   COVID_STATE    int64         
 8   CONSUMO_COVID  int64         
 9   MONTH_LONG     object        
 10  TEMPERATURE    float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(6), object(2)
memory usage: 275.4+ MB


In [176]:
data_barcelona['CONSUMO_COVID'].value_counts()

0    2793473
1     345930
Name: CONSUMO_COVID, dtype: int64

## BLOCK 2: CONSUMPTION BY ACTIVITY DATASET

Then, we have a dataset about the activities and the consumption of each. We will clean the dataset for then analyse the data.

### CONSUMPTION BY ACTIVITY DATASET

In [177]:
# open actividad por zona
data_actividad_barcelona = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/water_consumption/activity_barcelona.csv")

In [178]:
# delete null rows
data_actividad_barcelona=data_actividad_barcelona.dropna(how='any', axis=0)

# append new column LOCATION
data_actividad_barcelona['LOCATION'] = ['BARCELONA' for i in range(len(data_actividad_barcelona))]

# convert date object to datetime
data_actividad_barcelona['FECHA'] = pd.to_datetime(data_actividad_barcelona['FECHA'], infer_datetime_format=True)

# append year column
data_actividad_barcelona['YEAR'] = [date.strftime("%Y") for date in data_actividad_barcelona['FECHA']]

# append month column
data_actividad_barcelona['MONTH'] = [date.month for date in data_actividad_barcelona['FECHA']]

In [179]:
data_actividad_barcelona['YEAR'] = data_actividad_barcelona['YEAR'].astype(int)

In [180]:
# append new column TEMPERATURE
data_actividad_barcelona = data_actividad_barcelona.merge(temperature_barcelona, left_on=['MONTH', 'YEAR'], right_on=['Mes', 'Any'])
data_actividad_barcelona = data_actividad_barcelona.drop(['Any', 'Mes'], axis=1)

In [181]:
# delete unnamed: 0 column
data_actividad_barcelona = data_actividad_barcelona.drop(['Unnamed: 0'], axis=1)

In [182]:
# rename column names
data_actividad_barcelona.rename(columns = {'Desc_Mes':'MONTH_LONG', 'Temperatura':'TEMPERATURE'}, inplace = True)

In [183]:
print('There are ', len(data_actividad_barcelona['DESCP EPIGRAF'].unique()), ' activities')
print('Activities: ', data_actividad_barcelona['DESCP EPIGRAF'].unique())

There are  196  activities
Activities:  ['* Venda de pa, pans especials y pastisseria'
 'Salons i instituts de bellesa i gabinets estètica'
 'Comerç al detall de vins i begudes'
 'Serveis de publicitat, relacions públiques i simil'
 'Explotació electrònica per compte de tercers'
 "* d'articles de ferreteria, regal..."
 '* al detall de tota classe de penyores per el'
 '* de traducció i similars' 'Dtrs.,llicen.cienc. polítiques i socials'
 'Altres serveis de telecomunicacions'
 'Altres serveis privats de telecomunicació'
 'Comerç al detall de medicaments, productes sanitar'
 'Serveis en cafès i bars, amb i sense menjar'
 'Agents representants i corr. assegurances'
 "Agències d'assegurances i corredors assegurances"
 '* de formació i perfec profess superior' '* de bicicletes'
 'Salons de perruqueria i instituts de bellesa' 'ACTIVITAT DESCONEGUDA'
 "SUBMINISTRAMENT D'úS DOMèSTIC"
 '* de locals industrials i altres lloguers' 'COMUNICACIó EMA TARIFA C1A'
 'Altres professionals comerç i hotel

In [184]:
data_actividad_barcelona

,FECHA,CONSUMO,EPIGRAF,DESCP EPIGRAF,ID_CLIENTE,LOCATION,YEAR,MONTH,MONTH_LONG,TEMPERATURE
0,2019-01-01,45,I6442,"* Venda de pa, pans especials y pastisseria",29,BARCELONA,2019,1,Gener,8.1
1,2019-01-02,244,I6442,"* Venda de pa, pans especials y pastisseria",29,BARCELONA,2019,1,Gener,8.1
2,2019-01-03,261,I6442,"* Venda de pa, pans especials y pastisseria",29,BARCELONA,2019,1,Gener,8.1
3,2019-01-04,273,I6442,"* Venda de pa, pans especials y pastisseria",29,BARCELONA,2019,1,Gener,8.1
4,2019-01-05,212,I6442,"* Venda de pa, pans especials y pastisseria",29,BARCELONA,2019,1,Gener,8.1
...,...,...,...,...,...,...,...,...,...,...
1871195,2021-12-12,24,I8612,* de locals industrials i altres lloguers,132,BARCELONA,2021,12,Desembre,10.9
1871196,2021-12-13,135,I8332,* d'edificacions,132,BARCELONA,2021,12,Desembre,10.9
1871197,2021-12-13,135,I8612,* de locals industrials i altres lloguers,132,BARCELONA,2021,12,Desembre,10.9
1871198,2021-12-14,43,I8332,* d'edificacions,132,BARCELONA,2021,12,Desembre,10.9


In [185]:
data_actividad_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1871200 entries, 0 to 1871199
Data columns (total 10 columns):
 #   Column         Dtype         
---  ------         -----         
 0   FECHA          datetime64[ns]
 1   CONSUMO        int64         
 2   EPIGRAF        object        
 3   DESCP EPIGRAF  object        
 4   ID_CLIENTE     int64         
 5   LOCATION       object        
 6   YEAR           int32         
 7   MONTH          int64         
 8   MONTH_LONG     object        
 9   TEMPERATURE    float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(4)
memory usage: 149.9+ MB


In [186]:
# detect negative values of consumption
count_neg = detect_neg_values(data_actividad_barcelona, 'CONSUMO')
print('There are ', count_neg, ' negative values in consumption')
print('There are a total of ', len(data_actividad_barcelona), 'values of consumption')
print((count_neg/len(data_actividad_barcelona)) *100)

# so we can delete the negative values -> small part

There are  2328  negative values in consumption
There are a total of  1871200 values of consumption
0.12441214194100043


In [187]:
data_actividad_barcelona = data_actividad_barcelona[data_actividad_barcelona['CONSUMO']>=0]

## BLOCK 3: CENSUS SECTION DATASET

Then, we will clean the dataset about consumption by census section and population by consus section.

### POPULATION BY CENSUS SECTION DATASET

In [30]:
# open population by census section https://opendata-ajuntament.barcelona.cat/data/ca/dataset/taula-map-scensal (2021 and 2020)
data_population_2020 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/population_census_section/2020_12_TAULA_MAP_SCENSAL.csv")
data_population_2021 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/population_census_section/2021_12_TAULA_MAP_SCENSAL.csv")

# concat datasets
data_population = pd.concat([data_population_2020, data_population_2021], axis=0)
data_population

,SECCIO_CENSAL,HOMES,DONES,EDAT_0_A_14,EDAT_15_A_24,EDAT_25_A_64,EDAT_65_A_MES,NACIONALS,COMUNITARIS,ESTRANGERS,DATA_DADES
0,1001,708,599,140,104,860,203,687,148,472,06-12-2020 11:00:04
1,1002,790,707,149,184,966,198,589,198,710,06-12-2020 11:00:04
2,1003,2032,1640,707,497,2071,397,1851,284,1537,06-12-2020 11:00:04
3,1004,1651,1350,359,324,2018,300,1514,345,1142,06-12-2020 11:00:04
4,1005,1308,1150,297,262,1578,321,1225,350,883,06-12-2020 11:00:04
...,...,...,...,...,...,...,...,...,...,...,...
1063,10143,851,964,223,196,1001,395,1447,64,304,05-12-2021 11:00:05
1064,10234,818,988,166,157,986,497,1534,69,203,05-12-2021 11:00:05
1065,10235,614,689,141,84,695,383,1072,78,153,05-12-2021 11:00:05
1066,10236,755,850,149,110,800,546,1284,105,216,05-12-2021 11:00:05


In [31]:
# delete null rows
data_population=data_population.dropna(how='any', axis=0)

# convert date object to datetime
data_population['DATA_DADES'] = pd.to_datetime(data_population['DATA_DADES'], infer_datetime_format=True)

# append year column
data_population['YEAR'] = [date.strftime("%Y") for date in data_population['DATA_DADES']]

In [32]:
data_population

,SECCIO_CENSAL,HOMES,DONES,EDAT_0_A_14,EDAT_15_A_24,EDAT_25_A_64,EDAT_65_A_MES,NACIONALS,COMUNITARIS,ESTRANGERS,DATA_DADES,YEAR
0,1001,708,599,140,104,860,203,687,148,472,2020-06-12 11:00:04,2020
1,1002,790,707,149,184,966,198,589,198,710,2020-06-12 11:00:04,2020
2,1003,2032,1640,707,497,2071,397,1851,284,1537,2020-06-12 11:00:04,2020
3,1004,1651,1350,359,324,2018,300,1514,345,1142,2020-06-12 11:00:04,2020
4,1005,1308,1150,297,262,1578,321,1225,350,883,2020-06-12 11:00:04,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
1063,10143,851,964,223,196,1001,395,1447,64,304,2021-05-12 11:00:05,2021
1064,10234,818,988,166,157,986,497,1534,69,203,2021-05-12 11:00:05,2021
1065,10235,614,689,141,84,695,383,1072,78,153,2021-05-12 11:00:05,2021
1066,10236,755,850,149,110,800,546,1284,105,216,2021-05-12 11:00:05,2021


In [33]:
data_population['YEAR'] = data_population['YEAR'].astype(int)

In [34]:
data_population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2136 entries, 0 to 1067
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   SECCIO_CENSAL  2136 non-null   int64         
 1   HOMES          2136 non-null   int64         
 2   DONES          2136 non-null   int64         
 3   EDAT_0_A_14    2136 non-null   int64         
 4   EDAT_15_A_24   2136 non-null   int64         
 5   EDAT_25_A_64   2136 non-null   int64         
 6   EDAT_65_A_MES  2136 non-null   int64         
 7   NACIONALS      2136 non-null   int64         
 8   COMUNITARIS    2136 non-null   int64         
 9   ESTRANGERS     2136 non-null   int64         
 10  DATA_DADES     2136 non-null   datetime64[ns]
 11  YEAR           2136 non-null   int32         
dtypes: datetime64[ns](1), int32(1), int64(10)
memory usage: 208.6 KB


### CONSUMPTION BY CENSUS SECTION DATASET

In [35]:
# open consum per secció censal
data_seccio_censal = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/water_consumption/ConsumPerSeccioCensal_.csv")

In [36]:
# delete null rows
data_seccio_censal=data_seccio_censal.dropna(how='any', axis=0)

# convert date object to datetime
data_seccio_censal['DIA'] = pd.to_datetime(data_seccio_censal['DIA'], infer_datetime_format=True)

# append year column
data_seccio_censal['YEAR'] = [date.strftime("%Y") for date in data_seccio_censal['DIA']]

# append month column
data_seccio_censal['MONTH'] = [date.month for date in data_seccio_censal['DIA']]

# append day column
data_seccio_censal['DAY'] = [date.day for date in data_seccio_censal['DIA']]

In [37]:
# remove the 4 first digits from census section
def remove_first_digits(number):
    elem = str(number)
    return elem[-4:]
data_seccio_censal['SECCIO_CENSAL'] = [remove_first_digits(data) for data in data_seccio_censal['SECCIO_CENSAL']]

data_seccio_censal

,DIA,CONSUM,SECCIO_CENSAL,YEAR,MONTH,DAY
0,2019-01-01,80,3030,2019,1,1
1,2019-01-01,143,1053,2019,1,1
2,2019-01-01,341,3011,2019,1,1
3,2019-01-01,317,3093,2019,1,1
4,2019-01-01,75,3019,2019,1,1
...,...,...,...,...,...,...
14286093,2021-12-14,547,2115,2021,12,14
14286094,2021-12-14,44770,5031,2021,12,14
14286095,2021-12-14,76300,7039,2021,12,14
14286096,2021-12-14,251,4026,2021,12,14


In [38]:
data_seccio_censal['SECCIO_CENSAL'] = data_seccio_censal['SECCIO_CENSAL'].astype(int)
data_seccio_censal['YEAR'] = data_seccio_censal['YEAR'].astype(int)

In [39]:
data_seccio_censal

,DIA,CONSUM,SECCIO_CENSAL,YEAR,MONTH,DAY
0,2019-01-01,80,3030,2019,1,1
1,2019-01-01,143,1053,2019,1,1
2,2019-01-01,341,3011,2019,1,1
3,2019-01-01,317,3093,2019,1,1
4,2019-01-01,75,3019,2019,1,1
...,...,...,...,...,...,...
14286093,2021-12-14,547,2115,2021,12,14
14286094,2021-12-14,44770,5031,2021,12,14
14286095,2021-12-14,76300,7039,2021,12,14
14286096,2021-12-14,251,4026,2021,12,14


In [40]:
data_seccio_censal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14286098 entries, 0 to 14286097
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   DIA            datetime64[ns]
 1   CONSUM         int64         
 2   SECCIO_CENSAL  int32         
 3   YEAR           int32         
 4   MONTH          int64         
 5   DAY            int64         
dtypes: datetime64[ns](1), int32(2), int64(3)
memory usage: 545.0 MB


In [41]:
print('There are ',len(data_seccio_censal['SECCIO_CENSAL'].unique()), ' seccions censals')
print(data_seccio_censal['SECCIO_CENSAL'].value_counts())

There are  661  seccions censals
3025    148694
2077    100803
1023     90928
2081     90206
3024     89767
         ...  
9021       213
8107       164
9033       145
5060       110
5048        74
Name: SECCIO_CENSAL, Length: 661, dtype: int64


In [42]:
# detect negative values of consumption
count_neg = detect_neg_values(data_seccio_censal, 'CONSUM')
print('There are ', count_neg, ' negative values in consumption')
print('There are a total of ', len(data_seccio_censal), 'values of consumption')
print((count_neg/len(data_seccio_censal)) *100)

# so we can delete the negative values -> small part

There are  10738  negative values in consumption
There are a total of  14286098 values of consumption
0.07516398109546778


In [43]:
data_seccio_censal = data_seccio_censal[data_seccio_censal['CONSUM']>=0]

#### Merge datasets

In [47]:
data_sc = data_seccio_censal.merge(data_population, left_on=['SECCIO_CENSAL', 'YEAR'], right_on=['SECCIO_CENSAL', 'YEAR'])
data_sc

,DIA,CONSUM,SECCIO_CENSAL,YEAR,MONTH,DAY,HOMES,DONES,EDAT_0_A_14,EDAT_15_A_24,EDAT_25_A_64,EDAT_65_A_MES,NACIONALS,COMUNITARIS,ESTRANGERS,DATA_DADES
0,2020-01-01,233,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,2020-06-12 11:00:04
1,2020-01-01,178,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,2020-06-12 11:00:04
2,2020-01-01,138,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,2020-06-12 11:00:04
3,2020-01-01,438,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,2020-06-12 11:00:04
4,2020-01-01,1,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,2020-06-12 11:00:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7999037,2021-12-09,30950,7098,2021,12,9,1007,1146,302,236,1132,483,1986,61,106,2021-05-12 11:00:05
7999038,2021-12-10,28200,7098,2021,12,10,1007,1146,302,236,1132,483,1986,61,106,2021-05-12 11:00:05
7999039,2021-12-11,26800,7098,2021,12,11,1007,1146,302,236,1132,483,1986,61,106,2021-05-12 11:00:05
7999040,2021-12-12,16680,7098,2021,12,12,1007,1146,302,236,1132,483,1986,61,106,2021-05-12 11:00:05


In [48]:
# append new column TEMPERATURE
data_sc = data_sc.merge(temperature_barcelona, left_on=['MONTH', 'YEAR'], right_on=['Mes', 'Any'])
data_sc = data_sc.drop(['Any', 'Mes', 'DATA_DADES'], axis=1)
data_sc

,DIA,CONSUM,SECCIO_CENSAL,YEAR,MONTH,DAY,HOMES,DONES,EDAT_0_A_14,EDAT_15_A_24,EDAT_25_A_64,EDAT_65_A_MES,NACIONALS,COMUNITARIS,ESTRANGERS,Desc_Mes,Temperatura
0,2020-01-01,233,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,Gener,10.0
1,2020-01-01,178,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,Gener,10.0
2,2020-01-01,138,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,Gener,10.0
3,2020-01-01,438,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,Gener,10.0
4,2020-01-01,1,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,Gener,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7999037,2021-12-09,30950,7098,2021,12,9,1007,1146,302,236,1132,483,1986,61,106,Desembre,10.9
7999038,2021-12-10,28200,7098,2021,12,10,1007,1146,302,236,1132,483,1986,61,106,Desembre,10.9
7999039,2021-12-11,26800,7098,2021,12,11,1007,1146,302,236,1132,483,1986,61,106,Desembre,10.9
7999040,2021-12-12,16680,7098,2021,12,12,1007,1146,302,236,1132,483,1986,61,106,Desembre,10.9


In [49]:
data_sc['CONSUMPTION_PER_POPULATION'] = [data_sc['CONSUM'].iloc[i]/(data_sc['HOMES'].iloc[i]+data_sc['DONES'].iloc[i]) for i in range(len(data_sc))]

In [50]:
data_sc

,DIA,CONSUM,SECCIO_CENSAL,YEAR,MONTH,DAY,HOMES,DONES,EDAT_0_A_14,EDAT_15_A_24,EDAT_25_A_64,EDAT_65_A_MES,NACIONALS,COMUNITARIS,ESTRANGERS,Desc_Mes,Temperatura,CONSUMPTION_PER_POPULATION
0,2020-01-01,233,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,Gener,10.0,0.109802
1,2020-01-01,178,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,Gener,10.0,0.083883
2,2020-01-01,138,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,Gener,10.0,0.065033
3,2020-01-01,438,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,Gener,10.0,0.206409
4,2020-01-01,1,2166,2020,1,1,1016,1106,246,174,1300,402,1491,223,408,Gener,10.0,0.000471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7999037,2021-12-09,30950,7098,2021,12,9,1007,1146,302,236,1132,483,1986,61,106,Desembre,10.9,14.375290
7999038,2021-12-10,28200,7098,2021,12,10,1007,1146,302,236,1132,483,1986,61,106,Desembre,10.9,13.098003
7999039,2021-12-11,26800,7098,2021,12,11,1007,1146,302,236,1132,483,1986,61,106,Desembre,10.9,12.447747
7999040,2021-12-12,16680,7098,2021,12,12,1007,1146,302,236,1132,483,1986,61,106,Desembre,10.9,7.747329


In [51]:
data_sc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7999042 entries, 0 to 7999041
Data columns (total 18 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   DIA                         datetime64[ns]
 1   CONSUM                      int64         
 2   SECCIO_CENSAL               int32         
 3   YEAR                        int32         
 4   MONTH                       int64         
 5   DAY                         int64         
 6   HOMES                       int64         
 7   DONES                       int64         
 8   EDAT_0_A_14                 int64         
 9   EDAT_15_A_24                int64         
 10  EDAT_25_A_64                int64         
 11  EDAT_65_A_MES               int64         
 12  NACIONALS                   int64         
 13  COMUNITARIS                 int64         
 14  ESTRANGERS                  int64         
 15  Desc_Mes                    object        
 16  Temperatura       

## BLOCK 4: ELECTRICITY AND CONSUMPTION POSTAL CODE DATASETS

In this part we will clean the postal code dataset and merge it with the electricity by postal code dataset.

### ELECTRICITY BY POSTAL CODE DATASET

In [7]:
# open electricity dataset https://opendata-ajuntament.barcelona.cat/data/es/dataset/consum-electricitat-bcn
electr_2019 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/electricity_consumption/2019_consum_electricitat_bcn.csv")
electr_2020 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/electricity_consumption/2020_consum_electricitat_bcn.csv")
elctr_2021 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/electricity_consumption/2021_consum_electricitat_bcn.csv")

In [8]:
data_electr = pd.concat([electr_2019, electr_2020, elctr_2021], axis=0)
data_electr

,Any,Data,Codi_Postal,Sector_Economic,Tram_Horari,Valor
0,2019,2019-01-01,8001,Indústria,De 00:00:00 a 05:59:59 h,727
1,2019,2019-01-01,8001,Indústria,De 06:00:00 a 11:59:59 h,855
2,2019,2019-01-01,8001,Indústria,De 12:00:00 a 17:59:59 h,1175
3,2019,2019-01-01,8001,Indústria,De 18:00:00 a 23:59:59 h,1214
4,2019,2019-01-01,8001,Indústria,No consta,106
...,...,...,...,...,...,...
243640,2021,2021-12-31,8042,Serveis,De 00:00:00 a 05:59:59 h,22381
243641,2021,2021-12-31,8042,Serveis,De 06:00:00 a 11:59:59 h,41636
243642,2021,2021-12-31,8042,Serveis,De 12:00:00 a 17:59:59 h,42009
243643,2021,2021-12-31,8042,Serveis,De 18:00:00 a 23:59:59 h,39131


In [9]:
# delete null rows
data_electr=data_electr.dropna(how='any', axis=0)

# convert date object to datetime
data_electr['Data'] = pd.to_datetime(data_electr['Data'], infer_datetime_format=True)

In [10]:
# group by economic sector, postal code and data
data_electr = data_electr.groupby(['Data', 'Codi_Postal', 'Sector_Economic', 'Any']).sum('Valor')
data_electr = data_electr.reset_index()
data_electr

,Data,Codi_Postal,Sector_Economic,Any,Valor
0,2019-01-01,8001,Indústria,2019,4077
1,2019-01-01,8001,Residencial,2019,156961
2,2019-01-01,8001,Serveis,2019,214748
3,2019-01-01,8002,Indústria,2019,11294
4,2019-01-01,8002,No especificat,2019,30
...,...,...,...,...,...
146820,2021-12-31,8041,Residencial,2021,152575
146821,2021-12-31,8041,Serveis,2021,86508
146822,2021-12-31,8042,Indústria,2021,3375
146823,2021-12-31,8042,Residencial,2021,124800


In [11]:
# append month column
data_electr['MONTH'] = [date.month for date in data_electr['Data']]

# append day column
data_electr['DAY'] = [date.day for date in data_electr['Data']]

In [12]:
# separar en sector industria, residencial, serveis i no especificat
data_industria = data_electr[data_electr['Sector_Economic'] == 'Indústria']
data_residencial = data_electr[data_electr['Sector_Economic'] == 'Residencial']
data_serveis = data_electr[data_electr['Sector_Economic'] == 'Serveis']
data_na = data_electr[data_electr['Sector_Economic'] == 'No especificat']

In [13]:
data_industria

,Data,Codi_Postal,Sector_Economic,Any,Valor,MONTH,DAY
0,2019-01-01,8001,Indústria,2019,4077,1,1
3,2019-01-01,8002,Indústria,2019,11294,1,1
7,2019-01-01,8003,Indústria,2019,11999,1,1
11,2019-01-01,8004,Indústria,2019,167601,1,1
14,2019-01-01,8005,Indústria,2019,15091,1,1
...,...,...,...,...,...,...,...
146810,2021-12-31,8038,Indústria,2021,144387,12,31
146813,2021-12-31,8039,Indústria,2021,219870,12,31
146816,2021-12-31,8040,Indústria,2021,178926,12,31
146819,2021-12-31,8041,Indústria,2021,3184,12,31


### POSTAL CODE DATASET

In [14]:
# open postal code dataset
data_postal_code = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/water_consumption/ConsumPerCP.csv")

In [15]:
# delete null rows
data_postal_code=data_postal_code.dropna(how='any', axis=0)

# convert date object to datetime
data_postal_code['DIA'] = pd.to_datetime(data_postal_code['DIA'], infer_datetime_format=True)

# append year column
data_postal_code['YEAR'] = [date.strftime("%Y") for date in data_postal_code['DIA']]

# append month column
data_postal_code['MONTH'] = [date.month for date in data_postal_code['DIA']]

# append day column
data_postal_code['DAY'] = [date.day for date in data_postal_code['DIA']]

In [16]:
data_postal_code['YEAR'] = data_postal_code['YEAR'].astype(int)

In [17]:
# append new column TEMPERATURE
data_postal_code = data_postal_code.merge(temperature_barcelona, left_on=['MONTH', 'YEAR'], right_on=['Mes', 'Any'])
data_postal_code = data_postal_code.drop(['Any', 'Mes'], axis=1)

In [18]:
data_postal_code

,DIA,COD_POST_ADRE,CONSUM,YEAR,MONTH,DAY,Desc_Mes,Temperatura
0,2021-04-14,8036,597,2021,4,14,Abril,12.9
1,2021-04-03,8004,408,2021,4,3,Abril,12.9
2,2021-04-01,8027,1831,2021,4,1,Abril,12.9
3,2021-04-05,8754,516,2021,4,5,Abril,12.9
4,2021-04-07,8918,5702,2021,4,7,Abril,12.9
...,...,...,...,...,...,...,...,...
92547,2019-03-24,8020,298,2019,3,24,Març,13.5
92548,2019-03-13,8008,811,2019,3,13,Març,13.5
92549,2019-03-13,8912,9892,2019,3,13,Març,13.5
92550,2019-03-24,8010,883,2019,3,24,Març,13.5


In [19]:
data_postal_code.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92552 entries, 0 to 92551
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DIA            92552 non-null  datetime64[ns]
 1   COD_POST_ADRE  92552 non-null  int64         
 2   CONSUM         92552 non-null  int64         
 3   YEAR           92552 non-null  int32         
 4   MONTH          92552 non-null  int64         
 5   DAY            92552 non-null  int64         
 6   Desc_Mes       92552 non-null  object        
 7   Temperatura    92552 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(4), object(1)
memory usage: 6.0+ MB


In [20]:
# detect negative values of consumption
count_neg = detect_neg_values(data_postal_code, 'CONSUM')
print('There are ', count_neg, ' negative values in consumption')
print('There are a total of ', len(data_postal_code), 'values of consumption')
print((count_neg/len(data_postal_code)) *100)

# so we can delete the negative values -> small part

There are  180  negative values in consumption
There are a total of  92552 values of consumption
0.19448526233900942


In [21]:
data_postal_code = data_postal_code[data_postal_code['CONSUM']>=0]

#### Merge datasets

In [22]:
# industrial
data_postal_code_industrial = data_postal_code.merge(data_industria, left_on=['YEAR', 'MONTH', 'DAY', 'COD_POST_ADRE'], right_on=['Any', 'MONTH', 'DAY', 'Codi_Postal'])

# residencial
data_postal_code_residencial = data_postal_code.merge(data_residencial, left_on=['YEAR', 'MONTH', 'DAY', 'COD_POST_ADRE'], right_on=['Any', 'MONTH', 'DAY', 'Codi_Postal'])

# serveis
data_postal_code_serveis = data_postal_code.merge(data_serveis, left_on=['YEAR', 'MONTH', 'DAY', 'COD_POST_ADRE'], right_on=['Any', 'MONTH', 'DAY', 'Codi_Postal'])

# na
data_postal_code_na = data_postal_code.merge(data_na, left_on=['YEAR', 'MONTH', 'DAY', 'COD_POST_ADRE'], right_on=['Any', 'MONTH', 'DAY', 'Codi_Postal'])

In [23]:
# delete some repeated columns
data_postal_code_industrial = data_postal_code_industrial.drop(['Codi_Postal', 'Data', 'Any'], axis=1)
data_postal_code_residencial = data_postal_code_residencial.drop(['Codi_Postal', 'Data', 'Any'], axis=1)
data_postal_code_serveis = data_postal_code_serveis.drop(['Codi_Postal', 'Data', 'Any'], axis=1)
data_postal_code_na = data_postal_code_na.drop(['Codi_Postal', 'Data', 'Any'], axis=1)

In [24]:
data_postal_code_industrial

,DIA,COD_POST_ADRE,CONSUM,YEAR,MONTH,DAY,Desc_Mes,Temperatura,Sector_Economic,Valor
0,2021-04-14,8036,597,2021,4,14,Abril,12.9,Indústria,9487
1,2021-04-03,8004,408,2021,4,3,Abril,12.9,Indústria,181905
2,2021-04-01,8027,1831,2021,4,1,Abril,12.9,Indústria,4800
3,2021-04-05,8031,1046,2021,4,5,Abril,12.9,Indústria,1723
4,2021-04-15,8014,440,2021,4,15,Abril,12.9,Indústria,13766
...,...,...,...,...,...,...,...,...,...,...
45201,2019-03-13,8005,4257,2019,3,13,Març,13.5,Indústria,37666
45202,2019-03-13,8018,1614,2019,3,13,Març,13.5,Indústria,66164
45203,2019-03-24,8020,298,2019,3,24,Març,13.5,Indústria,46120
45204,2019-03-13,8008,811,2019,3,13,Març,13.5,Indústria,8603


In [25]:
data_postal_code_industrial.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45206 entries, 0 to 45205
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DIA              45206 non-null  datetime64[ns]
 1   COD_POST_ADRE    45206 non-null  int64         
 2   CONSUM           45206 non-null  int64         
 3   YEAR             45206 non-null  int32         
 4   MONTH            45206 non-null  int64         
 5   DAY              45206 non-null  int64         
 6   Desc_Mes         45206 non-null  object        
 7   Temperatura      45206 non-null  float64       
 8   Sector_Economic  45206 non-null  object        
 9   Valor            45206 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(5), object(2)
memory usage: 3.6+ MB


In [26]:
data_postal_code_residencial

,DIA,COD_POST_ADRE,CONSUM,YEAR,MONTH,DAY,Desc_Mes,Temperatura,Sector_Economic,Valor
0,2021-04-14,8036,597,2021,4,14,Abril,12.9,Residencial,93768
1,2021-04-03,8004,408,2021,4,3,Abril,12.9,Residencial,113620
2,2021-04-01,8027,1831,2021,4,1,Abril,12.9,Residencial,173196
3,2021-04-05,8031,1046,2021,4,5,Abril,12.9,Residencial,112638
4,2021-04-15,8014,440,2021,4,15,Abril,12.9,Residencial,198209
...,...,...,...,...,...,...,...,...,...,...
45201,2019-03-13,8005,4257,2019,3,13,Març,13.5,Residencial,213290
45202,2019-03-13,8018,1614,2019,3,13,Març,13.5,Residencial,199105
45203,2019-03-24,8020,298,2019,3,24,Març,13.5,Residencial,138470
45204,2019-03-13,8008,811,2019,3,13,Març,13.5,Residencial,52578


In [27]:
data_postal_code_residencial.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45206 entries, 0 to 45205
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DIA              45206 non-null  datetime64[ns]
 1   COD_POST_ADRE    45206 non-null  int64         
 2   CONSUM           45206 non-null  int64         
 3   YEAR             45206 non-null  int32         
 4   MONTH            45206 non-null  int64         
 5   DAY              45206 non-null  int64         
 6   Desc_Mes         45206 non-null  object        
 7   Temperatura      45206 non-null  float64       
 8   Sector_Economic  45206 non-null  object        
 9   Valor            45206 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(5), object(2)
memory usage: 3.6+ MB


In [28]:
data_postal_code_serveis

,DIA,COD_POST_ADRE,CONSUM,YEAR,MONTH,DAY,Desc_Mes,Temperatura,Sector_Economic,Valor
0,2021-04-14,8036,597,2021,4,14,Abril,12.9,Serveis,211917
1,2021-04-03,8004,408,2021,4,3,Abril,12.9,Serveis,122288
2,2021-04-01,8027,1831,2021,4,1,Abril,12.9,Serveis,171707
3,2021-04-05,8031,1046,2021,4,5,Abril,12.9,Serveis,50108
4,2021-04-15,8014,440,2021,4,15,Abril,12.9,Serveis,287422
...,...,...,...,...,...,...,...,...,...,...
45201,2019-03-13,8005,4257,2019,3,13,Març,13.5,Serveis,394243
45202,2019-03-13,8018,1614,2019,3,13,Març,13.5,Serveis,370508
45203,2019-03-24,8020,298,2019,3,24,Març,13.5,Serveis,229921
45204,2019-03-13,8008,811,2019,3,13,Març,13.5,Serveis,138830


In [29]:
data_postal_code_serveis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45206 entries, 0 to 45205
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DIA              45206 non-null  datetime64[ns]
 1   COD_POST_ADRE    45206 non-null  int64         
 2   CONSUM           45206 non-null  int64         
 3   YEAR             45206 non-null  int32         
 4   MONTH            45206 non-null  int64         
 5   DAY              45206 non-null  int64         
 6   Desc_Mes         45206 non-null  object        
 7   Temperatura      45206 non-null  float64       
 8   Sector_Economic  45206 non-null  object        
 9   Valor            45206 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(5), object(2)
memory usage: 3.6+ MB


## BLOCK 5: POPULATION AND CONSUMPTION DISTRICT ZONE DATASETS

Now we will focus on district zone in Barcelona. We have 4 datasets: -population by district zone -consumption by district zone -gini index by district zone -average rent by district zone.

We will clean the datasets and marge each with consumption dataset.

### POPULATION BY DISTRICT ZONE DATASET

In [52]:
# open population district dataset https://opendata-ajuntament.barcelona.cat/data/es/dataset/est-padro-nacionalitat-sexe
data_district_2019 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/population_district/2019_padro_nacionalitat_per_sexe.csv")
data_district_2020 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/population_district/2020_padro_nacionalitat_per_sexe.csv")
data_district_2021 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/population_district/2021_padro_nacionalitat_per_sexe.csv")

In [53]:
data_district_population = pd.concat([data_district_2019, data_district_2020, data_district_2021], axis=0)

In [54]:
data_district_population

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Sexe,Nacionalitat,Nombre
0,2019,1,Ciutat Vella,1,el Raval,Home,Espanya,11515
1,2019,1,Ciutat Vella,2,el Barri Gòtic,Home,Espanya,4166
2,2019,1,Ciutat Vella,3,la Barceloneta,Home,Espanya,4447
3,2019,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Home,Espanya,6167
4,2019,2,Eixample,5,el Fort Pienc,Home,Espanya,11426
...,...,...,...,...,...,...,...,...
8171,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,Homes,Apàtrides / No consta,0
8172,2021,10,Sant Martí,70,el Besòs i el Maresme,Homes,Apàtrides / No consta,2
8173,2021,10,Sant Martí,71,Provençals del Poblenou,Homes,Apàtrides / No consta,5
8174,2021,10,Sant Martí,72,Sant Martí de Provençals,Homes,Apàtrides / No consta,2


In [55]:
data_district_population['Sexe'].unique()

array(['Home', 'Dona', 'Dones', 'Homes'], dtype=object)

In [56]:
# delete null rows
data_district_population=data_district_population.dropna(how='any', axis=0)

In [57]:
# convert district majus
data_district_population['Nom_Districte'] = [data.upper() for data in data_district_population['Nom_Districte']]

In [61]:
# normalize text
data_district_population['Nom_Districte'] = [strip_accents(data) for data in data_district_population['Nom_Districte']]

In [62]:
# change nacionality to 'espanyol' or 'extranger' or 'NA'
data_district_population['Nacionalitat'] = [('Espanyol' if data_district_population['Nacionalitat'].iloc[row] =='Espanya' else ('NA' if data_district_population['Nacionalitat'].iloc[row] == 'Apàtrides / No consta' else 'Extranger')) for row in range(len(data_district_population['Nacionalitat'])) ]

data_district_population['Nacionalitat'].unique()

array(['Extranger'], dtype=object)

In [63]:
data_district_population = data_district_population.drop(['Codi_Barri', 'Codi_Districte'], axis=1)

In [64]:
data_grouped = data_district_population.groupby(['Nom_Districte', 'Any', 'Sexe', 'Nacionalitat']).sum('Nombre')

In [65]:
data_district_population = data_grouped.reset_index()

In [66]:
data_district_population

,Nom_Districte,Any,Sexe,Nacionalitat,Nombre
0,CIUTAT VELLA,2019,Dona,Extranger,49713
1,CIUTAT VELLA,2019,Home,Extranger,56107
2,CIUTAT VELLA,2020,Dona,Extranger,50134
3,CIUTAT VELLA,2020,Home,Extranger,58197
4,CIUTAT VELLA,2021,Dones,Extranger,64202
5,CIUTAT VELLA,2021,Homes,Extranger,65561
6,EIXAMPLE,2019,Dona,Extranger,143483
7,EIXAMPLE,2019,Home,Extranger,125612
8,EIXAMPLE,2020,Dona,Extranger,144056
9,EIXAMPLE,2020,Home,Extranger,126638


In [67]:
data_district_total = data_district_population.groupby(['Nom_Districte', 'Any']).sum('Nombre')
data_district_total = data_district_total.reset_index()
data_district_total

,Nom_Districte,Any,Nombre
0,CIUTAT VELLA,2019,105820
1,CIUTAT VELLA,2020,108331
2,CIUTAT VELLA,2021,129763
3,EIXAMPLE,2019,269095
4,EIXAMPLE,2020,270694
5,EIXAMPLE,2021,148776
6,GRACIA,2019,122853
7,GRACIA,2020,123651
8,GRACIA,2021,101953
9,HORTA-GUINARDO,2019,172473


In [68]:
data_district_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Nom_Districte  30 non-null     object
 1   Any            30 non-null     int64 
 2   Nombre         30 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 848.0+ bytes


### DISTRICT DATASET

In [69]:
# open district dataset
data_district = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/water_consumption/ConsumPerDistricte.csv")

In [70]:
# delete null rows
data_district=data_district.dropna(how='any', axis=0)

# convert date object to datetime
data_district['DIA'] = pd.to_datetime(data_district['DIA'], infer_datetime_format=True)

# append year column
data_district['YEAR'] = [date.strftime("%Y") for date in data_district['DIA']]

# append month column
data_district['MONTH'] = [date.month for date in data_district['DIA']]

# append day column
data_district['DAY'] = [date.day for date in data_district['DIA']]

data_district['YEAR'] = data_district['YEAR'].astype(int)

In [71]:
data_district

,DIA,DISTRICTE,CONSUM,YEAR,MONTH,DAY
0,2019-01-01,SANT BOI,11669,2019,1,1
1,2019-01-01,BELLVITGE - POLIGON GORNAL,374,2019,1,1
2,2019-01-01,SANT ANDREU,1890,2019,1,1
3,2019-01-01,CIUTAT VELLA,1890,2019,1,1
4,2019-01-01,GRACIA,1890,2019,1,1
...,...,...,...,...,...,...
44277,2021-12-14,BADALONA,631,2021,12,14
44278,2021-12-14,SANT ANDREU,798,2021,12,14
44279,2021-12-14,VILADECANS,505,2021,12,14
44280,2021-12-14,EIXAMPLE,798,2021,12,14


In [72]:
data_district.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44282 entries, 0 to 44281
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   DIA        44282 non-null  datetime64[ns]
 1   DISTRICTE  44282 non-null  object        
 2   CONSUM     44282 non-null  int64         
 3   YEAR       44282 non-null  int32         
 4   MONTH      44282 non-null  int64         
 5   DAY        44282 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(3), object(1)
memory usage: 1.9+ MB


In [73]:
# detect negative values of consumption
count_neg = detect_neg_values(data_district, 'CONSUM')
print('There are ', count_neg, ' negative values in consumption')
print('There are a total of ', len(data_district), 'values of consumption')
print((count_neg/len(data_district)) *100)

# so we can delete the negative values -> small part

There are  227  negative values in consumption
There are a total of  44282 values of consumption
0.5126236394020144


In [74]:
data_district = data_district[data_district['CONSUM']>=0]

#### Merge datasets

In [75]:
data_district_population = data_district.merge(data_district_total, left_on=['YEAR', 'DISTRICTE'], right_on=['Any', 'Nom_Districte'])

In [76]:
# delete null rows
data_district_population=data_district_population.dropna(how='any', axis=0)

In [77]:
# delete columns
data_district_population = data_district_population.drop(['Any', 'Nom_Districte'], axis=1)

In [78]:
# append a new column rate consumption per number of population
data_district_population['CONSUMPTION_PER_POPULATION'] = [data_district_population['CONSUM'].iloc[i]/data_district_population['Nombre'].iloc[i] for i in range(len(data_district_population))]

In [79]:
data_district_population

,DIA,DISTRICTE,CONSUM,YEAR,MONTH,DAY,Nombre,CONSUMPTION_PER_POPULATION
0,2019-01-01,SANT ANDREU,1890,2019,1,1,150264,0.012578
1,2019-01-02,SANT ANDREU,1835,2019,1,2,150264,0.012212
2,2019-01-03,SANT ANDREU,1812,2019,1,3,150264,0.012059
3,2019-01-04,SANT ANDREU,1813,2019,1,4,150264,0.012065
4,2019-01-05,SANT ANDREU,1704,2019,1,5,150264,0.011340
...,...,...,...,...,...,...,...,...
8491,2021-12-10,SANT ANDREU,796,2021,12,10,205410,0.003875
8492,2021-12-11,SANT ANDREU,763,2021,12,11,205410,0.003715
8493,2021-12-12,SANT ANDREU,727,2021,12,12,205410,0.003539
8494,2021-12-13,SANT ANDREU,779,2021,12,13,205410,0.003792


In [80]:
data_district_population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8496 entries, 0 to 8495
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   DIA                         8496 non-null   datetime64[ns]
 1   DISTRICTE                   8496 non-null   object        
 2   CONSUM                      8496 non-null   int64         
 3   YEAR                        8496 non-null   int32         
 4   MONTH                       8496 non-null   int64         
 5   DAY                         8496 non-null   int64         
 6   Nombre                      8496 non-null   int64         
 7   CONSUMPTION_PER_POPULATION  8496 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(4), object(1)
memory usage: 564.2+ KB


### AVERAGE RENT DATASET

In [81]:
# open average rent dataset
data_rent_2019 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/rent_district/2019_renda_neta_mitjana_per_llar.csv")
data_rent_2020 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/rent_district/2020_renda_neta_mitjana_per_llar.csv")

data_district_rent = pd.concat([data_rent_2019, data_rent_2020], axis=0)

# delete columns
data_district_rent = data_district_rent.drop(['Codi_Districte', 'Codi_Barri', 'Nom_Barri', 'Seccio_Censal'], axis=1)

# delete null rows
data_district_rent=data_district_rent.dropna(how='any', axis=0)

data_district_rent

,Any,Nom_Districte,Import_Euros
0,2019,Ciutat Vella,27387
1,2019,Ciutat Vella,23230
2,2019,Ciutat Vella,25565
3,2019,Ciutat Vella,28470
4,2019,Ciutat Vella,23125
...,...,...,...
1063,2020,Sant Martí,31951
1064,2020,Sant Martí,38690
1065,2020,Sant Martí,28857
1066,2020,Sant Martí,32129


In [82]:
data_district_rent = data_district_rent.groupby(['Any', 'Nom_Districte']).mean().round(2)
data_district_rent = data_district_rent.reset_index()

# convert district majus
data_district_rent['Nom_Districte'] = [data.upper() for data in data_district_rent['Nom_Districte']]

# normalize text
data_district_rent['Nom_Districte'] = [strip_accents(data) for data in data_district_rent['Nom_Districte']]

data_district_rent

,Any,Nom_Districte,Import_Euros
0,2019,CIUTAT VELLA,28441.74
1,2019,EIXAMPLE,44714.05
2,2019,GRACIA,41807.57
3,2019,HORTA-GUINARDO,35955.49
4,2019,LES CORTS,54430.02
5,2019,NOU BARRIS,30575.32
6,2019,SANT ANDREU,36717.19
7,2019,SANT MARTI,37820.86
8,2019,SANTS-MONTJUIC,35258.97
9,2019,SARRIA-SANT GERVASI,68802.73


In [83]:
data_district_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Any            20 non-null     int64  
 1   Nom_Districte  20 non-null     object 
 2   Import_Euros   20 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 608.0+ bytes


#### Merge datasets

In [84]:
data_rent = data_district.merge(data_district_rent, left_on=['YEAR', 'DISTRICTE'], right_on=['Any', 'Nom_Districte'])

In [85]:
data_rent = data_rent.merge(data_district_total, left_on=['YEAR', 'DISTRICTE'], right_on=['Any', 'Nom_Districte'])

In [86]:
# append a new column rate consumption per number of population
data_rent['RENT_PER_POPULATION'] = [data_rent['Import_Euros'].iloc[i]/data_rent['Nombre'].iloc[i] for i in range(len(data_rent))]

In [88]:
# delete columns
data_rent = data_rent.drop(['Any_x', 'Nom_Districte_x', 'Nom_Districte_y', 'Any_y'], axis=1)

In [89]:
data_rent

,DIA,DISTRICTE,CONSUM,YEAR,MONTH,DAY,Import_Euros,Nombre,RENT_PER_POPULATION
0,2019-01-01,SANT ANDREU,1890,2019,1,1,36717.19,150264,0.244351
1,2019-01-02,SANT ANDREU,1835,2019,1,2,36717.19,150264,0.244351
2,2019-01-03,SANT ANDREU,1812,2019,1,3,36717.19,150264,0.244351
3,2019-01-04,SANT ANDREU,1813,2019,1,4,36717.19,150264,0.244351
4,2019-01-05,SANT ANDREU,1704,2019,1,5,36717.19,150264,0.244351
...,...,...,...,...,...,...,...,...,...
5463,2020-12-26,SANT MARTI,701,2020,12,26,37713.01,241691,0.156038
5464,2020-12-27,SANT MARTI,673,2020,12,27,37713.01,241691,0.156038
5465,2020-12-28,SANT MARTI,732,2020,12,28,37713.01,241691,0.156038
5466,2020-12-30,SANT MARTI,719,2020,12,30,37713.01,241691,0.156038


In [90]:
data_rent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5468 entries, 0 to 5467
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DIA                  5468 non-null   datetime64[ns]
 1   DISTRICTE            5468 non-null   object        
 2   CONSUM               5468 non-null   int64         
 3   YEAR                 5468 non-null   int32         
 4   MONTH                5468 non-null   int64         
 5   DAY                  5468 non-null   int64         
 6   Import_Euros         5468 non-null   float64       
 7   Nombre               5468 non-null   int64         
 8   RENT_PER_POPULATION  5468 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int32(1), int64(4), object(1)
memory usage: 405.8+ KB


### GINI INDEX DATASET

In [91]:
# open gini index dataset
data_gini_2019 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/gini_index/2019_atles_renda_index_gini.csv")
data_gini_2020 = pd.read_csv("C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/original_datasets/gini_index/2020_atles_renda_index_gini.csv")

data_district_gini = pd.concat([data_gini_2019, data_gini_2020], axis=0)

# delete columns
data_district_gini = data_district_gini.drop(['Codi_Districte', 'Codi_Barri', 'Nom_Barri', 'Seccio_Censal'], axis=1)

# delete null rows
data_district_gini=data_district_gini.dropna(how='any', axis=0)

data_district_gini

,Any,Nom_Districte,Index_Gini
0,2019,Ciutat Vella,35.6
1,2019,Ciutat Vella,38.2
2,2019,Ciutat Vella,36.8
3,2019,Ciutat Vella,37.1
4,2019,Ciutat Vella,37.1
...,...,...,...
1063,2020,Sant Martí,28.3
1064,2020,Sant Martí,26.2
1065,2020,Sant Martí,31.4
1066,2020,Sant Martí,26.8


In [92]:
data_district_gini = data_district_gini.groupby(['Any', 'Nom_Districte']).mean().round(2)
data_district_gini = data_district_gini.reset_index()

# convert district majus
data_district_gini['Nom_Districte'] = [data.upper() for data in data_district_gini['Nom_Districte']]

# normalize text
data_district_gini['Nom_Districte'] = [strip_accents(data) for data in data_district_gini['Nom_Districte']]

data_district_gini

,Any,Nom_Districte,Index_Gini
0,2019,CIUTAT VELLA,36.00
1,2019,EIXAMPLE,34.67
2,2019,GRACIA,33.04
3,2019,HORTA-GUINARDO,30.23
4,2019,LES CORTS,33.96
5,2019,NOU BARRIS,29.18
6,2019,SANT ANDREU,28.86
7,2019,SANT MARTI,30.07
8,2019,SANTS-MONTJUIC,31.51
9,2019,SARRIA-SANT GERVASI,39.01


In [93]:
data_district_gini.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Any            20 non-null     int64  
 1   Nom_Districte  20 non-null     object 
 2   Index_Gini     20 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 608.0+ bytes


#### Merge datasets

In [94]:
data_gini = data_district.merge(data_district_gini, left_on=['YEAR', 'DISTRICTE'], right_on=['Any', 'Nom_Districte'])
data_gini

,DIA,DISTRICTE,CONSUM,YEAR,MONTH,DAY,Any,Nom_Districte,Index_Gini
0,2019-01-01,SANT ANDREU,1890,2019,1,1,2019,SANT ANDREU,28.86
1,2019-01-02,SANT ANDREU,1835,2019,1,2,2019,SANT ANDREU,28.86
2,2019-01-03,SANT ANDREU,1812,2019,1,3,2019,SANT ANDREU,28.86
3,2019-01-04,SANT ANDREU,1813,2019,1,4,2019,SANT ANDREU,28.86
4,2019-01-05,SANT ANDREU,1704,2019,1,5,2019,SANT ANDREU,28.86
...,...,...,...,...,...,...,...,...,...
5463,2020-12-26,SANT MARTI,701,2020,12,26,2020,SANT MARTI,30.35
5464,2020-12-27,SANT MARTI,673,2020,12,27,2020,SANT MARTI,30.35
5465,2020-12-28,SANT MARTI,732,2020,12,28,2020,SANT MARTI,30.35
5466,2020-12-30,SANT MARTI,719,2020,12,30,2020,SANT MARTI,30.35


In [95]:
# delete columns
data_gini = data_gini.drop(['Nom_Districte', 'Any'], axis=1)

In [96]:
data_gini

,DIA,DISTRICTE,CONSUM,YEAR,MONTH,DAY,Index_Gini
0,2019-01-01,SANT ANDREU,1890,2019,1,1,28.86
1,2019-01-02,SANT ANDREU,1835,2019,1,2,28.86
2,2019-01-03,SANT ANDREU,1812,2019,1,3,28.86
3,2019-01-04,SANT ANDREU,1813,2019,1,4,28.86
4,2019-01-05,SANT ANDREU,1704,2019,1,5,28.86
...,...,...,...,...,...,...,...
5463,2020-12-26,SANT MARTI,701,2020,12,26,30.35
5464,2020-12-27,SANT MARTI,673,2020,12,27,30.35
5465,2020-12-28,SANT MARTI,732,2020,12,28,30.35
5466,2020-12-30,SANT MARTI,719,2020,12,30,30.35


In [97]:
data_gini.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5468 entries, 0 to 5467
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DIA         5468 non-null   datetime64[ns]
 1   DISTRICTE   5468 non-null   object        
 2   CONSUM      5468 non-null   int64         
 3   YEAR        5468 non-null   int32         
 4   MONTH       5468 non-null   int64         
 5   DAY         5468 non-null   int64         
 6   Index_Gini  5468 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(1)
memory usage: 320.4+ KB


In [99]:
# save clean datasets

temperature_barcelona.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/temperature_barcelona.csv', index=False)

#block1
#data_barcelona.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block1/data_water_global.csv', index=False)
#data_industrial_barcelona.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block1/data_industrial_water.csv', index=False)
#data_comercial_barcelona.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block1/data_commercial_water.csv', index=False)
#data_domestic_barcelona.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block1/data_domestic_water.csv', index=False)

#block2
#data_actividad_barcelona.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block2/data_activity_water.csv', index=False)

#block3
data_sc.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block3/data_water_population_census_section.csv', index=False)
data_seccio_censal.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block3/data_water_census_section.csv', index=False)

#block4
data_postal_code_residencial.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block4/data_residencial_postal_code.csv', index=False)
data_postal_code_industrial.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block4/data_industrial_postal_code.csv', index=False)
data_postal_code_serveis.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block4/data_services_postal_code.csv', index=False)

#block5
data_rent.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block5/data_rent_disctrict.csv', index=False)
data_gini.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block5/data_gini_district.csv', index=False)
data_district_population.to_csv('C:/Users/mirei/OneDrive/Escritorio/TFG/datasets/clean_datasets/block5/data_population_district.csv', index=False)